In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import pandas as pd
import numpy
import plotly.graph_objects as go

In [3]:
pd.options.display.max_columns = None
pd.options.display.max_rows = 500

In [4]:
root = "yb_hack"
sys.path.append(os.getcwd()[:os.getcwd().find(root)+len(root)])

In [5]:
BATH_PATH= r'C:\\Users\\Swiss SDI\\OneDrive\\Bureau\\hackathon\\yb_hack\\data\\Daten'

In [6]:
lz_path = os.path.join(BATH_PATH, '0_Lagerzugänge 110_200_10_01.01.20_31.05.20.xlsx')
lz = pd.read_excel(io=lz_path, header=2)
la_path = os.path.join(BATH_PATH, '0_Lagerabgänge 110_200_10_01.01.20_31.05.20.xlsx')
la = pd.read_excel(io=la_path, header=2)
sorten_path = os.path.join(BATH_PATH, '0_Sorten_und_Sortengruppen.xlsx')
sorten = pd.read_excel(io=sorten_path, header=0)
transport_path = os.path.join(BATH_PATH, '0_Transportaufträge 110 010120_310520.xlsx')
transport = pd.read_excel(io=transport_path, header=2)

In [7]:
lz.head(1)

,Gruppe,Sorte,Sortenbezeichnung,Menge,Preis,ME,Kosten,Wert,L-Datum,Wiege-Nr.,Konto,Beleg-Nr,Lager,Abt,PC,Waggon/KFZ,Kontrakt,SB,WIP,Scheck
0,105.0,11161.0,Schwerschrott 8s2,0.0,0.0,to,0.0,-510.2,2020-01-01,0.0,9110610.0,130050808,200.0,110.0,0.0,NaN,NaN,DOG,NaN,0.0


In [8]:
la.head(1)

,Gruppe,Sorte,Sortenbezeichnung,Menge,Preis,ME,Kosten,Wert,L-Datum,Wiege-Nr.,Konto,Beleg-Nr,Lager,Abt,PC,Waggon/KFZ,Kontrakt,SB,WIP,Scheck
0,100.0,10000.0,Leichteisen / Shreddervormaterial,0,0.0,to,0.0,11.8,2020-01-01,1.0,9110620.0,130050609,200.0,110.0,0.0,NaN,NaN,ANA,NaN,0.0


In [9]:
sorten.head(1)

,Gruppe,Gruppe Kurzname,Sorte,Kurzname,Bezeichnung,Sortenklasse,Sparte,Mengeneinheit,Umrechnungsfaktor Einkauf,je ME,Umrechnungsfaktor Verkauf,je ME.1
0,100,SHREDDERPR,10000,LEICHTEISE,Leichteisen / Shreddervormaterial,Bestandsführung auf Gruppenebene,FE-Schrott,to,0,NaN,0,NaN


In [10]:
transport.head(1)

,Datum,Wiegenr,Fahrauftrag,Fahrzeug,Hänger,Behälter Anz,Anteil,Größe,m3/t,Anfahrstelle,Ziel,Sorte,Bezeichnung,EAS,Menge (t),Istkosten,Plankosten,Istkosten pro To,Plankosten pro To,Abweichung pro To
0,"[100006] ELREC AG, Trimmis",NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0


# Analyse Stock

In [11]:
incoming = lz.loc[lz["ME"] == "to", ["Sorte", "Menge"]].groupby("Sorte").sum()
incoming.reset_index(inplace=True)
outcoming = la.loc[la["ME"] == "to", ["Sorte", "Menge"]].groupby("Sorte").sum()
outcoming.reset_index(inplace=True)

In [12]:
def find_thommen(x):
    if "Thommen" in str(x):
        return True
    else:
        return False

transport["thommen_out"] = transport["Anfahrstelle"].apply(lambda x: find_thommen(x))
transport["thommen_in"] = transport["Ziel"].apply(lambda x: find_thommen(x))

In [13]:
outcoming_transport = transport.loc[transport["thommen_out"] == True, 
                                    ["Sorte", "Menge (t)"]].groupby("Sorte").sum()
outcoming_transport["Menge_outcoming_transport"] = outcoming_transport["Menge (t)"]*1000
outcoming_transport.reset_index(inplace=True)

In [14]:
incoming_transport = transport.loc[transport["thommen_in"] == True, 
                                    ["Sorte", "Menge (t)"]].groupby("Sorte").sum()
incoming_transport["Menge_incoming_transport"] = incoming_transport["Menge (t)"]*1000
incoming_transport.reset_index(inplace=True)

In [15]:
merge = incoming.merge(outcoming, how="outer", on="Sorte", suffixes=('_incoming', '_outcoming'))
merge = merge.merge(outcoming_transport, how="outer", on="Sorte")
merge = merge.merge(incoming_transport, how="outer", on="Sorte")
merge = merge.fillna(0)

In [16]:
merge["incoming"] = merge["Menge_incoming"] + merge["Menge_incoming_transport"]
merge["outcoming"] = merge["Menge_outcoming"] + merge["Menge_outcoming_transport"]
merge["diff"] = merge["incoming"] - merge["outcoming"]

In [17]:
merge[["Sorte", "incoming", "outcoming", "diff"]]

,Sorte,incoming,outcoming,diff
0,10000.0,32302802.1,2609715.0,29693087.1
1,10005.0,206901.5,15366.0,191535.5
2,10010.0,5703077.0,38710.0,5664367.0
3,10011.0,208583.0,0.0,208583.0
4,10012.0,1940.0,0.0,1940.0
5,10030.0,279380.0,0.0,279380.0
6,10031.0,7404.0,0.0,7404.0
7,10040.0,374055.0,0.0,374055.0
8,10050.0,396197.0,0.0,396197.0
9,10053.0,45.0,0.0,45.0


In [18]:
merge

,Sorte,Menge_incoming,Menge_outcoming,Menge (t)_x,Menge_outcoming_transport,Menge (t)_y,Menge_incoming_transport,incoming,outcoming,diff
0,10000.0,22042649.1,19027.0,2590.688,2590688.0,10260.153,10260153.0,32302802.1,2609715.0,29693087.1
1,10005.0,170025.5,0.0,15.366,15366.0,36.876,36876.0,206901.5,15366.0,191535.5
2,10010.0,5569467.0,1910.0,36.800,36800.0,133.610,133610.0,5703077.0,38710.0,5664367.0
3,10011.0,200383.0,0.0,0.000,0.0,8.200,8200.0,208583.0,0.0,208583.0
4,10012.0,1940.0,0.0,0.000,0.0,0.000,0.0,1940.0,0.0,1940.0
5,10030.0,279380.0,0.0,0.000,0.0,0.000,0.0,279380.0,0.0,279380.0
6,10031.0,5444.0,0.0,0.000,0.0,1.960,1960.0,7404.0,0.0,7404.0
7,10040.0,230107.0,0.0,0.000,0.0,143.948,143948.0,374055.0,0.0,374055.0
8,10050.0,395717.0,0.0,0.000,0.0,0.480,480.0,396197.0,0.0,396197.0
9,10053.0,45.0,0.0,0.000,0.0,0.000,0.0,45.0,0.0,45.0
